In [36]:
import requests
import json
# import openai
from openai import AzureOpenAI
import time
from dotenv import load_dotenv
import os, time
from tqdm import tqdm

load_dotenv()

True

In [2]:
os.getenv("DEPLOYMENT_NAME")

'R21-GPT4'

In [3]:
test = """I broke 4 ribs,I was given 30 TylenolW/Cod. It did absolutely nothing for pain. Ibuprofen gave me a tiny bit of relief.i was surprised."""

In [4]:
context = """You are an AI assistant that helps to annotate social media comment data related to opioids and/or drug addiction, and you know everything about identifying the grammartical person (ie first, second, or third person) of a sentence or post"""

In [6]:
prompt = """I will give you an opioid-related social media comment to annotate. You will return a onehot integer of length 3 (eg. 010). If the comment contains a first-person or personal account with opioids put 1 in position 1 of the onehot integer else 0. If the comment contains a second-person experience with opioid, put 1 in position 2 of the onehot integer else 0. If the comment contains a third-person experience with opioid, put 1 in position 3 of the onehot integer else 0. Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. Note that some comments can have multiple persons-perspective and you will put 1 in the positions corresponding to the persons-perspective present. Example return 110 for 'I lost a partner. I am sorry to hear you lost someone too. My condolences to you and your family', and 011 for 'You should take lower dose. They should be careful with supplies from the street'. 
Annotate the following:\n"""

In [7]:
print(prompt)

I will give you an opioid-related social media comment to annotate. You will return a onehot integer of length 3 (eg. 010). If the comment contains a first-person or personal account with opioids put 1 in position 1 of the onehot integer else 0. If the comment contains a second-person experience with opioid, put 1 in position 2 of the onehot integer else 0. If the comment contains a third-person experience with opioid, put 1 in position 3 of the onehot integer else 0. Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. Note that some comments can have multiple persons-perspective and you will put 1 in the positions corresponding to the persons-perspective present. Example return 110 for 'I lost a partner. I am sorry to hear you lost someone too. My condolences to you and your family', and 011 for 'You should take lower dose. They should be careful with supplies from the street'. 


In [11]:
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-01"
)

response = client.chat.completions.create(
    model=os.getenv("DEPLOYMENT_NAME"), # model = "deployment_name".
    messages=[
        {"role": "system", "content": context},
        {"role": "user", "content": prompt},
    ]
)

print(response.choices[0].message.content)

100


In [12]:
def query(context, prompt):
    client = AzureOpenAI(
      azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
      api_key=os.getenv("AZURE_OPENAI_KEY"),  
      api_version="2024-02-01"
    )
    
    response = client.chat.completions.create(
        model=os.getenv("DEPLOYMENT_NAME"), # model = "deployment_name".
        messages=[
            {"role": "system", "content": context},
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content

# Persons

In [14]:
context = """You are an AI assistant that helps to annotate social media comment data related to opioids and/or drug addiction, and you know everything about identifying the grammartical person (ie first, second, or third person) of a sentence or post"""

In [28]:
prompt_template = """I will give you an opioid-related social media comment to annotate. You will return a onehot integer of length 3 (eg. 010). If the comment contains a first-person or personal account with opioids put 1 in position 1 of the onehot integer else 0. If the comment contains a second-person experience with opioid, put 1 in position 2 of the onehot integer else 0. If the comment contains a third-person experience with opioid, put 1 in position 3 of the onehot integer else 0. Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. Note that some comments can have multiple persons-perspective and you will put 1 in the positions corresponding to the persons-perspective present. Example return 110 for 'I lost a partner. I am sorry to hear you lost someone too. My condolences to you and your family', and 011 for 'You should take lower dose. They should be careful with supplies from the street'. 
Annotate the following (only return the annotation):\n"""

In [29]:
with open("./persons.txt") as f:
    comments = f.read()
    comments = comments.split("\n\n")

In [ ]:
for i in tqdm(range(len(comments))):
    comment = comments[i]
    text = ''
    for _ in range(20):
        prompt = prompt_template + comment
        response = query(context, prompt)
        text += response + "\n"
        time.sleep(0.5)
    with open(f"./persons_predictions/sample_{i}.txt", "w") as f:
        f.write(text)

# Expert

In [42]:
context = "You are an AI assistant that helps to annotate social media comment data related to opioids and/or drug addiction, and you can differentiate between comments generated by experts and those that are not"

In [39]:
prompt_template = """I will give you an opioid-related social media comment to annotate. Return 1 if the post is written by an expert and 0 if it is not. An expert here is broadly defined as someone who works in healthcare. Some posts have hints that the author works in healthcare. Example: 'Neurosurgeon here, I see alot of patients', 'I am a physician assistant, and I have seen a lot of that', 'My patients respond well to XYZ drug'. Post that don't have hint that the author work in healthcare is not considered generated by an expert. Example: '8 months clean now', 'I am so sorry to hear this', etc. Return 1 for generated comments, 0 otherwise.
Annotate the following:\n"""

In [40]:
with open("./experts.txt") as f:
    comments = f.read()
    comments = comments.split("\n\n")

In [41]:
text = ""
for i in tqdm(range(len(comments))):
    comment = comments[i]
    prompt = prompt_template + comment
    response = query(context, prompt)
    text += response +
    time.sleep(0.5)

print(comments)

 96%|█████████▋| 27/28 [00:35<00:01,  1.31s/it]


KeyboardInterrupt: 

In [43]:
text

''